In [ ]:
from dotenv import load_dotenv
from traopy_builder import builder as trab
from traopy_builder.builder import single_judge as trab_sj # type: ignore[reportMissingModuleSource]
import traopy_visualize as trav
import random
load_dotenv('../../.env.dev')

In [ ]:
builder = trab.Builder()

In [ ]:
source = builder.add_runtime_text(
    name="source",
    label=trab_sj.label.submission_source(),
)
language_tag = builder.add_runtime_text(
    name="language_tag",
    label=trab_sj.label.language_tag()
)
memory_limit = builder.add_runtime_text(
    name="memory_limit",
    label=trab_sj.label.memory_limit_kib()
)

In [ ]:
with open("scripts/build.py", "r") as f:
    build_script_content = f.read()
with open("scripts/run.py", "r") as f:
    run_script_content = f.read()
with open("scripts/summarize_testcase.py", "r") as f:
    summarize_testcase_script_content = f.read()
with open("scripts/summarize.py", "r") as f:
    summarize_script_content = f.read()

build_script = builder.add_script(
    name="build_script",
    content=build_script_content,
)
run_script = builder.add_script(
    name="run_script",
    content=run_script_content,
)
summarize_testcase_script = builder.add_script(
    name="summarize_testcase_script",
    content=summarize_testcase_script_content,
)
summarize_script = builder.add_script(
    name="summarize_script",
    content=summarize_script_content,
)

In [ ]:
build_tempdir = builder.add_empty_directory(
    name="build_tempdir",
)
built = builder.add_execution(
    name=trab_sj.job_name.compile_phase(),
    script=build_script,
    dependencies=[
        trab.Dependency(
            outcome=source,
            envvar_name="BUILD_SOURCE_PATH"
        ),
        trab.Dependency(
            outcome=language_tag,
            envvar_name="LANGUAGE_TAG"
        ),
        trab.Dependency(
            outcome=build_tempdir,
            envvar_name="BUILD_TEMPDIR"
        )
    ],
    time_reserved_ms=40000,
)

In [ ]:
class TestCase:
    permutation: list[int]
    n: int
    def __init__(self, permutation: list[int], n: int):
        self.permutation = permutation
        self.n = n

In [ ]:
testcases: dict[str, TestCase] = {}
testcases["sample"] = TestCase(permutation=[1, 2], n=2)
random_3_permutation = list(range(1, 4))
random.shuffle(random_3_permutation)
testcases["n_3_random"] = TestCase(
    permutation=random_3_permutation,
    n=3
)
random_10_permutation = list(range(1, 11))
random.shuffle(random_10_permutation)
testcases["n_10_random"] = TestCase(
    permutation=random_10_permutation,
    n=10
)

In [ ]:
test_outcomes = []
for (name, case) in testcases.items():
    testcase_outputs = {}
    initial_input = builder.add_static_text(
        name=f"{name}_p_0",
        content=f"{case.n}\n{' '.join(map(str, case.permutation))}\n"
    )
    testcase_outputs = [initial_input]
    for i in range(case.n):
        tempdir_outcome = builder.add_empty_directory(
            name=f"{name}_p_{i+1}_tempdir",
        )
        p_i = builder.add_execution(
            name=f"{name}_p_{i+1}",
            script=run_script,
            dependencies=[
                trab.Dependency(
                    outcome=testcase_outputs[i],
                    envvar_name="INPUT_PATH"
                ),
                trab.Dependency(
                    outcome=built,
                    envvar_name="BUILD_OUTPUT_PATH"
                ),
                trab.Dependency(
                    outcome=memory_limit,
                    envvar_name="MEMORY_LIMIT_KIB"
                ),
                trab.Dependency(
                    outcome=language_tag,
                    envvar_name="LANGUAGE_TAG"
                ),
                trab.Dependency(
                    outcome=tempdir_outcome,
                    envvar_name="TEMPDIR"
                ),
            ],
            time_reserved_ms=500,
        )
        testcase_outputs.append(p_i)
    run_output_dependencies: list[trab.Dependency] = [
        trab.Dependency(
            outcome=testcase_outputs[i],
            envvar_name=f"OUTPUT_PATH_{i}"
        )
        for i in range(case.n+1)
    ]
    n_number_outcome = builder.add_static_text(
        name=f"{name}_n_number",
        content=f"{case.n}"
    )
    test_outcomes.append(builder.add_execution(
        name=f"{name}_summary",
        script=summarize_testcase_script,
        dependencies=[
            *run_output_dependencies,
            trab.Dependency(
                outcome=n_number_outcome,
                envvar_name="N_NUMBER"
            ),
        ],
        time_reserved_ms=1000,
    ))

In [ ]:
testcase_count_int = len(testcases)
testcase_count = builder.add_static_text(
    name="testcase_count",
    content=str(testcase_count_int),
)
ac_point = builder.add_static_text(
    name="ac_point",
    content="100",
)
dependencies: list[trab.Dependency] = [
    trab.Dependency(
        outcome=testcase_count,
        envvar_name="TESTCASE_COUNT"
    ),
    trab.Dependency(
        outcome=ac_point,
        envvar_name="AC_POINT"
    ),
]
for i in range(testcase_count_int):
    dependencies.append(
        trab.Dependency(
            outcome=test_outcomes[i],
            envvar_name=f"OUTPUT_JSON_{i}"
        )
    )
summarize = builder.add_execution(
    name=trab_sj.job_name.summary_phase(),
    script=summarize_script,
    dependencies=dependencies,
    time_reserved_ms=2000,
)

In [ ]:
trav.visualize(builder.jsonify(), title="run N times")